In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from datetime import datetime, date, time

st.set_page_config(page_title="AI Study Scheduler", layout="wide")

st.markdown(
    "<h1 style='text-align:center;'>⏰ AI Study Scheduler</h1>",
    unsafe_allow_html=True
)
st.markdown(
    "<p style='text-align:center;'>Add tasks → AI prioritizes → Smart scheduling</p>",
    unsafe_allow_html=True
)

st.divider()

if "tasks" not in st.session_state:
    st.session_state.tasks = []


st.sidebar.header("Add Task")

task_name = st.sidebar.text_input("Task Name")

hardness = st.sidebar.selectbox(
    "Hardness of Task",
    ["Easy", "Medium", "Hard"]
)

free_start = st.sidebar.time_input("Free Time Starts", time(9, 0))
free_end = st.sidebar.time_input("Free Time Ends", time(11, 30))

deadline_date = st.sidebar.date_input("Deadline Date")
deadline_time = st.sidebar.time_input("Deadline Time")

add_task = st.sidebar.button("➕ Add Task")

hardness_weight = {
    "Easy": 1,
    "Medium": 2,
    "Hard": 3
}


if add_task:
    if task_name.strip() == "":
        st.sidebar.warning("Please enter task name.")
    else:
        deadline_dt = datetime.combine(deadline_date, deadline_time)

        st.session_state.tasks.append({
            "Task": task_name,
            "Hardness": hardness,
            "Weight": hardness_weight[hardness],
            "Deadline": deadline_dt
        })

        st.sidebar.success("Task added successfully!")

if st.session_state.tasks:
    st.header("📋 AI-Managed Task List")

    df = pd.DataFrame(st.session_state.tasks)

    # Remaining time in HOURS
    df["Hours Remaining"] = (
        df["Deadline"] - datetime.now()
    ).dt.total_seconds() / 3600

    # Priority calculation
    df["Priority Score"] = df["Weight"] / df["Hours Remaining"]

    # Sort by priority
    df = df.sort_values(
        by="Priority Score",
        ascending=False
    )

    # Free time calculation
    start_dt = datetime.combine(date.today(), free_start)
    end_dt = datetime.combine(date.today(), free_end)
    free_hours = (end_dt - start_dt).total_seconds() / 3600

    total_weight = df["Weight"].sum()

    # Allocate time proportionally
    df["Allocated Study Time (hrs)"] = (
        df["Weight"] / total_weight
    ) * free_hours

    # Prepare display table
    display_df = df[[
        "Task",
        "Hardness",
        "Allocated Study Time (hrs)",
        "Hours Remaining"
    ]]

    display_df["Allocated Study Time (hrs)"] = display_df[
        "Allocated Study Time (hrs)"
    ].round(2)

    display_df["Hours Remaining"] = display_df[
        "Hours Remaining"
    ].round(2)

    st.dataframe(display_df, use_container_width=True)

    # AI Recommendation
    top_task = display_df.iloc[0]

    st.subheader("🧠 AI Recommendation")
    st.success(
        f"Start with **{top_task['Task']}** — "
        f"only **{top_task['Hours Remaining']} hours remaining** till deadline."
    )

else:
    st.info("👈 Add tasks from the sidebar to generate an AI schedule.")


Overwriting app.py
